In [36]:
import torch
from deepFake_model import DeepFakeDetectionModel
import os

In [13]:
x = torch.load('deepFakeDetection.pth')
model_args = {
    'n_layers': 5, 'n_hidden': 256, 'bias': True, 'drop_out': 0.1
    }

In [14]:
model = DeepFakeDetectionModel(**model_args)
model.load_state_dict(x['model'])


<All keys matched successfully>

In [48]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(55)
if torch.cuda.is_available():
    torch.cuda.manual_seed(55)



class CSVDataset(Dataset):
    def __init__(self, data):
        self.X = data.iloc[:, :-1].values 
        self.y = data.iloc[:, -1].values   

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32 , device = device)
        y = torch.tensor(self.y[idx], dtype=torch.float32 , device = device)
        return X, y


#csv_file =r"D:\projects\DeepFake-Arabic-Voice-Detection\NN\testing samples\REAL\mariamAR.csv"
#csv_file = r'D:\projects\DeepFake-Arabic-Voice-Detection\NN\testing samples\REAL\englishAudio.csv'
#csv_file = r'D:\projects\DeepFake-Arabic-Voice-Detection\NN\testing samples\FAKE\mariam_to_aboENG.csv'
#csv_file = r'D:\projects\DeepFake-Arabic-Voice-Detection\NN\testing samples\FAKE\mariam_to_margotAR.csv'
csv_file = r'D:\projects\DeepFake-Arabic-Voice-Detection\NN\testing samples\FAKE\mariam_to_nedalAR.csv'


data = pd.read_csv(csv_file)

inference_dataset = CSVDataset(data)
batch_size = 128
inference_dataloader = DataLoader(inference_dataset, batch_size=batch_size, shuffle=True)


In [49]:
model = model.to(device)
model.eval()

DeepFakeDetectionModel(
  (ST_ln): LayerNorm((26,), eps=1e-05, elementwise_affine=True)
  (ST_fc): Linear(in_features=26, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0-4): 5 x block(
      (ln): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (fc): Linear(in_features=256, out_features=256, bias=True)
      (relu): ReLU()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (LAST_fc): Linear(in_features=256, out_features=1, bias=True)
  (LAST_ln): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (sigmoid): Sigmoid()
)

In [50]:
inference_confusion_mat = np.zeros((2,2) , dtype=int)

for X , _ in inference_dataloader:
    
        
    if os.path.basename(os.path.dirname(csv_file)) == "FAKE":
        y = torch.ones((X.shape[0],1))
    elif os.path.basename(os.path.dirname(csv_file)) == "REAL":
        y = torch.zeros((X.shape[0],1))
    else:
        continue

    print("""
    TP | FN
    -------
    FP | TN
    """)

    
    
    
    outputs = model(X)
    
    
    preds = (outputs > 0.5).float().cpu().detach().numpy()
    inference_confusion_mat += confusion_matrix(y.cpu().detach().numpy() , preds)
    
    
    print(f"""train confusion matrix : {inference_confusion_mat[0]}   
                    {inference_confusion_mat[1]}""")
    print(f"train accuracy : {(inference_confusion_mat[0,0]+inference_confusion_mat[1,1]) / np.sum(inference_confusion_mat):.3f}")
        



    TP | FN
    -------
    FP | TN
    
train confusion matrix : [0 0]   
                    [ 5 55]
train accuracy : 0.917
